In [ ]:
#打群架的博弈策略分析

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
# 设置中文显示
plt.rc("font", family='Microsoft YaHei', weight="bold")
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
class CombatUnit:
    def __init__(self, attack, defense, impact):
        self.attack = attack    # 攻击强度
        self.defense = defense  # 防御强度
        self.impact = impact    # 体系影响值
        self.alive = True       # 存活状态
        self.damage_received = 0  # 当前回合受到的伤害
        
    def __repr__(self):
        return f"Unit(A:{self.attack:.1f}, D:{self.defense:.1f}, I:{self.impact:.1f}, {'Alive' if self.alive else 'Dead'})"
    
    def reset(self):
        self.alive = True
        self.damage_received = 0

class Army:
    def __init__(self, units, strategy='随机攻击', allocation_type='平衡型'):
        self.units = units
        self.strategy = strategy
        self.allocation_type = allocation_type
        
    @property
    def alive_units(self):
        return [u for u in self.units if u.alive]
    
    @property
    def total_impact(self):
        return sum(u.impact for u in self.units if u.alive)
    
    @property
    def total_attack(self):
        return sum(u.attack for u in self.alive_units)
    
    def reset(self):
        for unit in self.units:
            unit.reset()
    
    def choose_targets(self, enemy_army):
        """根据策略选择攻击目标"""
        alive_enemies = enemy_army.alive_units
        
        if not alive_enemies:
            return []
        
        # 随机攻击策略
        if self.strategy == '随机攻击':
            targets = np.random.choice(alive_enemies, size=len(self.alive_units), replace=True)
            return list(targets)
        
        # 集中攻击最高价值目标
        elif self.strategy == '集火高价值':
            max_impact = max(u.impact for u in alive_enemies)
            high_impact_targets = [u for u in alive_enemies if u.impact == max_impact]
            if len(high_impact_targets) > 1:
                target = np.random.choice(high_impact_targets)
            else:
                target = high_impact_targets[0]
            return [target] * len(self.alive_units)
        
        # 集中攻击最低防御目标
        elif self.strategy == '集火低防御':
            min_defense = min(u.defense for u in alive_enemies)
            low_defense_targets = [u for u in alive_enemies if u.defense == min_defense]
            if len(low_defense_targets) > 1:
                target = np.random.choice(low_defense_targets)
            else:
                target = low_defense_targets[0]
            return [target] * len(self.alive_units)
        
        # 分散攻击
        elif self.strategy == '分散攻击':
            # 尽可能均匀分配攻击目标
            targets = []
            for i, unit in enumerate(self.alive_units):
                target_idx = i % len(alive_enemies)
                targets.append(alive_enemies[target_idx])
            return targets
        
        # 混合策略：80%单位集火高价值，20%随机攻击
        elif self.strategy == '混合策略':
            main_target = max(alive_enemies, key=lambda u: u.impact)
            targets = []
            for i, unit in enumerate(self.alive_units):
                if i < 0.8 * len(self.alive_units):
                    targets.append(main_target)
                else:
                    targets.append(np.random.choice(alive_enemies))
            return targets
        
        # return np.random.choice(alive_enemies, size=len(self.alive_units), replace=True)

def create_army(total_points=1000, num_units=10, allocation_type='平衡型', strategy='随机攻击'):
    """创建军队并分配资源"""
    # 生成体系影响值（重要性不同）
    impacts = np.random.uniform(0.8, 1.2, num_units)
    impacts /= impacts.sum()  # 归一化
    
    points_per_unit = total_points / num_units
    units = []
    
    # 平衡分配：攻防平均
    if allocation_type == '平衡型':
        for i in range(num_units):
            attack = points_per_unit * 0.5
            defense = points_per_unit * 0.5
            units.append(CombatUnit(attack, defense, impacts[i]))
    
    # 攻击型分配
    elif allocation_type == '攻击型':
        for i in range(num_units):
            attack = points_per_unit * 0.7
            defense = points_per_unit * 0.3
            units.append(CombatUnit(attack, defense, impacts[i]))
    
    # 防御型分配
    elif allocation_type == '防御型':
        for i in range(num_units):
            attack = points_per_unit * 0.3
            defense = points_per_unit * 0.7
            units.append(CombatUnit(attack, defense, impacts[i]))
    
    # 特种分配：少数精英+多数普通
    elif allocation_type == '特种型':
        elite_ratio = 0.2  # 精英单位比例
        num_elite = max(1, int(num_units * elite_ratio))
        for i in range(num_units):
            if i < num_elite:  # 精英单位
                attack = points_per_unit * 1.8 * 0.6
                defense = points_per_unit * 1.8 * 0.4
            else:       # 普通单位
                attack = points_per_unit * 0.8 * 0.4
                defense = points_per_unit * 0.8 * 0.6
            units.append(CombatUnit(attack, defense, impacts[i]))
    
    # 混合分配：高低搭配
    elif allocation_type == '混合型':
        for i in range(num_units):
            if i % 3 == 0:  # 攻击型单位
                attack = points_per_unit * 0.8
                defense = points_per_unit * 0.2
            elif i % 3 == 1:  # 防御型单位
                attack = points_per_unit * 0.2
                defense = points_per_unit * 0.8
            else:  # 平衡型单位
                attack = points_per_unit * 0.5
                defense = points_per_unit * 0.5
            units.append(CombatUnit(attack, defense, impacts[i]))
    
    return Army(units, strategy, allocation_type)

def simulate_battle(army1, army2, max_rounds=20):
    """模拟一场战斗"""
    army1.reset()
    army2.reset()
    
    for round in range(max_rounds):
        # 双方选择目标
        targets1 = army1.choose_targets(army2)
        targets2 = army2.choose_targets(army1)
        
        # 重置伤害计数器
        for unit in army1.units + army2.units:
            unit.damage_received = 0
        
        # 军队1攻击
        for attacker, target in zip(army1.alive_units, targets1):
            target.damage_received += attacker.attack
        
        # 军队2攻击
        for attacker, target in zip(army2.alive_units, targets2):
            target.damage_received += attacker.attack
        
        # 检查目标是否被摧毁
        for target in set(targets1):
            if target.damage_received > target.defense:
                target.alive = False
                
        for target in set(targets2):
            if target.damage_received > target.defense:
                target.alive = False
        
        # 检查战斗是否结束
        if len(army1.alive_units) == 0 or len(army2.alive_units) == 0:
            break
    
    # 计算剩余影响力
    impact1 = army1.total_impact
    impact2 = army2.total_impact
    
    # 判断胜负
    if impact1 > impact2:
        # print ('军队1(%s, %s) vs 军队2(%s,%s), 军队1胜利'%(army1.allocation_type, army1.strategy, army2.allocation_type, army2.strategy))
        return 1  # 军队1胜利
    elif impact2 > impact1:
        # print ('军队1(%s, %s) vs 军队2(%s,%s), 军队2胜利'%(army1.allocation_type, army1.strategy, army2.allocation_type, army2.strategy))
        return -1  # 军队2胜利
    else:
        # print ('军队1(%s, %s) vs 军队2(%s,%s), 平局'%(army1.allocation_type, army1.strategy, army2.allocation_type, army2.strategy))
        return 0  # 平局

In [ ]:
# 实验1：不同策略组合的成功率比较（10个战斗单元）
def experiment1(num_units=10, num_simulations=100):
    allocation_types = ['平衡型', '攻击型', '防御型', '特种型', '混合型']
    attack_strategies = ['随机攻击', '集火高价值', '集火低防御', '分散攻击', '混合策略']
    
    # 存储结果
    # results = np.zeros((len(allocation_types), len(attack_strategies), len(allocation_types), len(attack_strategies)))
    results = np.zeros((len(allocation_types), len(attack_strategies)))
    
    # 进度条
    total_simulations = len(allocation_types) * len(attack_strategies) * len(allocation_types) * len(attack_strategies) * num_simulations
    pbar = tqdm(total=total_simulations, desc="实验1进度")
    
    for i, alloc1 in enumerate(allocation_types):
        for j, strat1 in enumerate(attack_strategies):
            # wins_i_j = 0
            wins = 0
            for k, alloc2 in enumerate(allocation_types):
                for l, strat2 in enumerate(attack_strategies):
                    # wins = 0
                    for _ in range(num_simulations):
                        army1 = create_army(num_units=num_units, allocation_type=alloc1, strategy=strat1)
                        army2 = create_army(num_units=num_units, allocation_type=alloc2, strategy=strat2)
                        result = simulate_battle(army1, army2)
                        if result == 1:
                            wins += 1
                        pbar.update(1)
                    # win_rate = wins / num_simulations
                    # results[i, j, k, l] = win_rate
            results[i, j] = wins / (len(allocation_types)*len(attack_strategies)*num_simulations)
                    
    pbar.close()
    
    return results, allocation_types, attack_strategies

In [ ]:
# 可视化函数
def plot_experiment1(results, allocation_types, attack_strategies, num_units):
    # 简化展示：固定敌方策略为平衡型+随机攻击时，不同策略的表现
    # enemy_alloc_idx = allocation_types.index('平衡型')
    # enemy_strat_idx = attack_strategies.index('随机攻击')
    
    # win_rates = results[:, :, enemy_alloc_idx, enemy_strat_idx]
    win_rates = results
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(win_rates, annot=True, fmt=".2f", cmap="coolwarm",
                xticklabels=attack_strategies, yticklabels=allocation_types)
    plt.title('双方各%d个作战单位，不同策略组合的平均胜率'%num_units, fontsize=16)
    plt.xlabel('攻击策略', fontsize=14)
    plt.ylabel('资源分配策略', fontsize=14)
    plt.tight_layout()
    plt.savefig('experiment1_heatmap-%d.png'%num_units, dpi=300)
    # plt.show()
    
    # 找出最佳策略组合
    best_idx = np.unravel_index(np.argmax(win_rates), win_rates.shape)
    best_alloc = allocation_types[best_idx[0]]
    best_strat = attack_strategies[best_idx[1]]
    best_rate = win_rates[best_idx]
    
    print(f"最佳策略组合: 资源分配={best_alloc}, 攻击策略={best_strat}, 胜率={best_rate:.2f}")

In [ ]:
def plot_unit_size_results_alloc(results, alloc_types, unit_sizes):
    """绘制不同作战单元数量下的资源分配方式胜率变化图"""
    plt.figure(figsize=(12, 8))
    
    # 定义颜色和标记
    colors = ['blue', 'red', 'green', 'purple', 'orange']
    markers = ['o', 's', '^', 'D', 'v']
    
    # 绘制每条线
    for i, alloc_type in enumerate(alloc_types):
        plt.plot(unit_sizes, results[i], 
                 color=colors[i], marker=markers[i], 
                 linewidth=2, markersize=8, 
                 label=alloc_type)
    
    # 设置图表属性
    plt.title('不同作战单元数量下各种资源分配方式的平均胜率', fontsize=16)
    plt.xlabel('作战单元数量', fontsize=14)
    plt.ylabel('平均胜率', fontsize=14)
    plt.legend(loc='best', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # 设置x轴刻度
    plt.xticks(unit_sizes)
    # 添加水平参考线
    plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.savefig('unit_size_vs_win_rate_alloc_types.png', dpi=300)
    # plt.show()
    
    # 打印最佳策略
    best_strategies = {}
    for i, size in enumerate(unit_sizes):
        best_idx = np.argmax(results[:, i])
        best_strategy = alloc_types[best_idx]
        best_win_rate = results[best_idx, i]
        best_strategies[size] = (best_strategy, best_win_rate)
        print(f"单元数量 {size}: 最佳策略={best_strategy}, 胜率={best_win_rate:.3f}")

In [ ]:
def plot_unit_size_results_strat(results, strat_types, unit_sizes):
    """绘制不同作战单元数量下的攻击方式胜率变化图"""
    plt.figure(figsize=(12, 8))
    
    # 定义颜色和标记
    colors = ['blue', 'red', 'green', 'purple', 'orange']
    markers = ['o', 's', '^', 'D', 'v']
    
    # 绘制每条线
    for i, strat_type in enumerate(strat_types):
        plt.plot(unit_sizes, results[i], 
                 color=colors[i], marker=markers[i], 
                 linewidth=2, markersize=8, 
                 label=strat_type)
    
    # 设置图表属性
    plt.title('不同作战单元数量下各种攻击方式的平均胜率', fontsize=16)
    plt.xlabel('作战单元数量', fontsize=14)
    plt.ylabel('平均胜率', fontsize=14)
    plt.legend(loc='best', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # 设置x轴刻度
    plt.xticks(unit_sizes)
    # 添加水平参考线
    plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.savefig('unit_size_vs_win_rate_strat_types.png', dpi=300)
    # plt.show()
    
    # 打印最佳策略
    best_strategies = {}
    for i, size in enumerate(unit_sizes):
        best_idx = np.argmax(results[:, i])
        best_strategy = strat_types[best_idx]
        best_win_rate = results[best_idx, i]
        best_strategies[size] = (best_strategy, best_win_rate)
        print(f"单元数量 {size}: 最佳策略={best_strategy}, 胜率={best_win_rate:.3f}")

In [ ]:
# 主程序
if __name__ == "__main__":
    #实验1：不同策略组合
    unit_sizes = [10,20,30,40,50,60,70,80,90,100]
    
    allocation_types = ['平衡型', '攻击型', '防御型', '特种型', '混合型']
    attack_strategies = ['随机攻击', '集火高价值', '集火低防御', '分散攻击', '混合策略']
    
    resutls_alloc_types = np.zeros((len(allocation_types), len(unit_sizes)))
    results_strat_types = np.zeros((len(attack_strategies), len(unit_sizes)))
    
    for i in range(len(unit_sizes)):
        print("开始实验1: 不同策略组合比较 (%d个战斗单元)"%unit_sizes[i])
        results1, alloc_types, strat_types = experiment1(num_units=unit_sizes[i], num_simulations=100)
        
        for j in range(len(alloc_types)):
            resutls_alloc_types[j, i] = np.mean(results1[j,:])

        for k in range(len(strat_types)):
            results_strat_types[k, i] = np.mean(results1[:,k])
            
        plot_experiment1(results1, alloc_types, strat_types, num_units=unit_sizes[i])

    plot_unit_size_results_alloc(resutls_alloc_types, allocation_types, unit_sizes)
    plot_unit_size_results_strat(results_strat_types, attack_strategies, unit_sizes)
    
    print("所有实验完成! 结果图表已保存")